In [23]:
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [24]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [25]:
dataset.drop('User ID', axis=1)


,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [26]:
dataset['Purchased'].value_counts()


Purchased
0    257
1    143
Name: count, dtype: int64

In [27]:
dataset.columns


Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [28]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [29]:
dep=dataset[['Purchased']]
dep


,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=0.30,random_state=0)

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import CategoricalNB
param_grid={'alpha':[0.1, 0.5, 1.0, 1.5, 2.0],'force_alpha':[True,False],'fit_prior':[True,False]}
grid=GridSearchCV(CategoricalNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [32]:
Y_pred=grid.predict(X_test)
Y_pred

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0], dtype=int64)

In [33]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

[[69 10]
 [11 30]]


In [34]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,Y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.86      0.87      0.87        79
           1       0.75      0.73      0.74        41

    accuracy                           0.82       120
   macro avg       0.81      0.80      0.80       120
weighted avg       0.82      0.82      0.82       120



In [35]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,Y_pred, average='macro')
f1_macro

0.8043326345213138

In [36]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print("roc_auc score for the best parameter {} {:.2f}".format(grid.best_params_,roc_score ))

roc_auc score for the best parameter {'alpha': 0.1, 'fit_prior': True, 'force_alpha': True} 0.88


In [22]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.061158,0.020349,0.032427,0.015178,0.1,True,True,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.753521,0.690193,0.730543,0.792857,NaN,NaN,NaN,1
1,0.035893,0.006891,0.016174,0.008087,0.1,True,False,"{'alpha': 0.1, 'fit_prior': True, 'force_alpha...",0.753521,0.690193,0.730543,0.792857,NaN,NaN,NaN,1
2,0.039549,0.007246,0.020122,0.011683,0.1,False,True,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
3,0.031572,0.004743,0.014148,0.008091,0.1,False,False,"{'alpha': 0.1, 'fit_prior': False, 'force_alph...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
4,0.030396,0.000059,0.020206,0.006399,0.5,True,True,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.769643,0.684601,0.742690,0.792857,NaN,NaN,NaN,1
5,0.032540,0.002600,0.018195,0.004052,0.5,True,False,"{'alpha': 0.5, 'fit_prior': True, 'force_alpha...",0.769643,0.684601,0.742690,0.792857,NaN,NaN,NaN,1
6,0.030298,0.003441,0.019039,0.010236,0.5,False,True,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.753521,0.684601,0.733523,0.855556,NaN,NaN,NaN,1
7,0.033580,0.005817,0.014208,0.005032,0.5,False,False,"{'alpha': 0.5, 'fit_prior': False, 'force_alph...",0.753521,0.684601,0.733523,0.855556,NaN,NaN,NaN,1
8,0.029541,0.008172,0.016239,0.008120,1.0,True,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.714286,0.692857,0.737557,0.809578,NaN,NaN,NaN,1
9,0.034874,0.005738,0.016235,0.005023,1.0,True,False,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.714286,0.692857,0.737557,0.809578,NaN,NaN,NaN,1
